In [1]:
import json
import pandas as pd
import tensorflow as tf

labelPath = "C:/Users/RA_CT/Desktop/Tek_Juice/cldModel/cassava-leaf-disease-classification/label_num_to_disease_map.json"

with open(labelPath, "r") as f:
    label_map = json.load(f)

print(label_map)

{'0': 'Cassava Bacterial Blight (CBB)', '1': 'Cassava Brown Streak Disease (CBSD)', '2': 'Cassava Green Mottle (CGM)', '3': 'Cassava Mosaic Disease (CMD)', '4': 'Healthy'}


In [2]:
# Load images and their labels 
trainLabel = "C:/Users/RA_CT/Desktop/Tek_Juice/cldModel/cassava-leaf-disease-classification/train.csv"

df = pd.read_csv(trainLabel)
print(df.head())

         image_id  label
0  1000015157.jpg      0
1  1000201771.jpg      3
2   100042118.jpg      1
3  1000723321.jpg      1
4  1000812911.jpg      3


In [3]:
# Build full image paths 
imagePathBase = "C:/Users/RA_CT/Desktop/Tek_Juice/cldModel/cassava-leaf-disease-classification/train_images"

image_paths = [imagePathBase + "/" + img for img in df["image_id"]]
labels = df["label"].values   # numeric labels

In [4]:
from sklearn.model_selection import train_test_split

train_paths, val_paths, train_labels, val_labels = train_test_split(
    image_paths,
    labels,
    test_size=0.1,
    stratify=labels,
    random_state=42
)

print(len(train_paths), "train images")
print(len(val_paths), "validation images")

19257 train images
2140 validation images


In [8]:
#  Preprocessing the images 

from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

IMAGE_SIZE = 224
BATCH_SIZE = 32

def process_image(path, label):
    img = tf.io.read_file(path)
    img = tf.io.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (IMAGE_SIZE, IMAGE_SIZE))
    img = preprocess_input(img)
    return img, label

In [6]:
# Build TF datasets (train + validation) 
train_ds = tf.data.Dataset.from_tensor_slices((train_paths, train_labels))
train_ds = train_ds.map(process_image).shuffle(2048).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

val_ds = tf.data.Dataset.from_tensor_slices((val_paths, val_labels))
val_ds = val_ds.map(process_image).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)


In [7]:
# Verify sample image 
for img, lbl in train_ds.take(1):
    print(img.shape, lbl.shape)
    break


(32, 224, 224, 3) (32,)


In [9]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

base_model = MobileNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)

base_model.trainable = False   

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dense(
    128,
    activation='relu',
    kernel_regularizer=tf.keras.regularizers.l2(1e-4)
)(x)
x = tf.keras.layers.Dropout(0.4)(x)
output = Dense(5, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


In [10]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)

Epoch 1/10
602/602 ━━━━━━━━━━━━━━━━━━━━ 446s 611ms/step - accuracy: 0.6811 - loss: 0.8811 - val_accuracy: 0.7234 - val_loss: 0.7701
Epoch 2/10
602/602 ━━━━━━━━━━━━━━━━━━━━ 354s 555ms/step - accuracy: 0.7244 - loss: 0.7637 - val_accuracy: 0.7037 - val_loss: 0.8947
Epoch 3/10
602/602 ━━━━━━━━━━━━━━━━━━━━ 375s 584ms/step - accuracy: 0.7389 - loss: 0.7237 - val_accuracy: 0.7238 - val_loss: 0.7588
Epoch 4/10
602/602 ━━━━━━━━━━━━━━━━━━━━ 351s 545ms/step - accuracy: 0.7510 - loss: 0.6842 - val_accuracy: 0.7257 - val_loss: 0.7599
Epoch 5/10
602/602 ━━━━━━━━━━━━━━━━━━━━ 370s 574ms/step - accuracy: 0.7571 - loss: 0.6587 - val_accuracy: 0.7355 - val_loss: 0.7365
Epoch 6/10
602/602 ━━━━━━━━━━━━━━━━━━━━ 368s 567ms/step - accuracy: 0.7733 - loss: 0.6263 - val_accuracy: 0.7318 - val_loss: 0.7738
Epoch 7/10
602/602 ━━━━━━━━━━━━━━━━━━━━ 364s 565ms/step - accuracy: 0.7810 - loss: 0.6018 - val_accuracy: 0.7299 - val_loss: 0.8145
Epoch 8/10
602/602 ━━━━━━━━━━━━━━━━━━━━ 367s 567ms/step - accuracy: 0.7917 -